# Imports

In [2]:
from pprint import pprint

import pandas as pd
import numpy as np
import nltk
import re

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Exercises
- Do your work for this exercise in a file named model.

- Take the work we did in the lessons further:

    - What other types of models (i.e. different classifcation algorithms) could you use?
    def clean(text: str) -> list:
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    text = (text.encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split() # tokenization
    return [wnl.lemmatize(word) for word in words if word not in stopwords]- How do the models compare when trained on term frequency data alone, instead of TF-IDF values?

In [3]:
def clean(text: str) -> list:
    'A simple function to cleanup text data'
    wnl = nltk.stem.WordNetLemmatizer()
    stopwords = set(nltk.corpus.stopwords.words('english'))
    text = (text.encode('ascii', 'ignore')
             .decode('utf-8', 'ignore')
             .lower())
    words = re.sub(r'[^\w\s]', '', text).split() # tokenization
    return [wnl.lemmatize(word) for word in words if word not in stopwords]

In [4]:
data = [
    'Python is pretty cool',
    'Python is a nice programming language with nice syntax',
    'I think SQL is cool too',
]

### Bag of Words

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
bag_of_words = cv.fit_transform(data)
bag_of_words

<3x12 sparse matrix of type '<class 'numpy.int64'>'
	with 16 stored elements in Compressed Sparse Row format>

In [6]:
pprint(data)
pd.DataFrame(bag_of_words.todense(), columns=cv.get_feature_names())

['Python is pretty cool',
 'Python is a nice programming language with nice syntax',
 'I think SQL is cool too']


,cool,is,language,nice,pretty,programming,python,sql,syntax,think,too,with
0,1,1,0,0,1,0,1,0,0,0,0,0
1,0,1,1,2,0,1,1,0,1,0,0,1
2,1,1,0,0,0,0,0,1,0,1,1,0


### TF-IDF

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidfs = tfidf.fit_transform(data)

pprint(data)
pd.DataFrame(tfidfs.todense(), columns=tfidf.get_feature_names())

['Python is pretty cool',
 'Python is a nice programming language with nice syntax',
 'I think SQL is cool too']


,cool,is,language,nice,pretty,programming,python,sql,syntax,think,too,with
0,0.480458,0.373119,0.000000,0.000000,0.631745,0.000000,0.480458,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.197673,0.334689,0.669378,0.000000,0.334689,0.254540,0.000000,0.334689,0.000000,0.000000,0.334689
2,0.383770,0.298032,0.000000,0.000000,0.000000,0.000000,0.000000,0.504611,0.000000,0.504611,0.504611,0.000000


### Bag Of Ngrams

In [8]:
#Asking to show bigrams because ngram_range=(2,2)
cv = CountVectorizer(ngram_range=(2, 2))
bag_of_words = cv.fit_transform(data)

In [9]:
pprint(data)
pd.DataFrame(bag_of_words.todense(), columns=cv.get_feature_names())

['Python is pretty cool',
 'Python is a nice programming language with nice syntax',
 'I think SQL is cool too']


,cool too,is cool,is nice,is pretty,language with,nice programming,nice syntax,pretty cool,programming language,python is,sql is,think sql,with nice
0,0,0,0,1,0,0,0,1,0,1,0,0,0
1,0,0,1,0,1,1,1,0,1,1,0,0,1
2,1,1,0,0,0,0,0,0,0,0,1,1,0


## Modeling

### Decision Tree Model

In [10]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier

df = pd.read_csv('spam_clean.csv')

In [11]:
cv = CountVectorizer()
#clean text and then join everything
X = cv.fit_transform(df.text.apply(clean).apply(' '.join))
y = df.label

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=12)

tree = DecisionTreeClassifier(max_depth=5)
tree.fit(X_train, y_train)


#accuracy score
tree.score(X_train, y_train)

0.9306708548350908

In [12]:
best_words = (
    # or, e.g. lm.coef_
    pd.Series(dict(zip(cv.get_feature_names(), tree.feature_importances_)))
    .sort_values()
    .tail(5)
    .index
)

In [13]:
best_words

Index(['later', 'claim', 'text', 'txt', 'call'], dtype='object')

### Random Forest Model

In [14]:
#Create the object
rf = RandomForestClassifier(bootstrap=True, 
                            class_weight=None, 
                            criterion='gini',
                            min_samples_leaf=3,
                            n_estimators=100,max_depth=3,random_state=123)


#Fit it
rf.fit(X_train, y_train)

                            

RandomForestClassifier(max_depth=3, min_samples_leaf=3, random_state=123)

In [15]:
#Run accuracy score                            
print('Accuracy of random forest classifier on training set: {:.2f}'.format(rf.score(X_train, y_train)))

Accuracy of random forest classifier on training set: 0.87


### KNN Model

In [16]:
#create KNN object
# weights = ['uniform', 'density']
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

In [17]:
#Fit it
knn.fit(X_train, y_train)

KNeighborsClassifier()

In [18]:
#Evaluate accuracy
print('Accuracy of KNN classifier on training set: {:.2f}'
     .format(knn.score(X_train, y_train)))

Accuracy of KNN classifier on training set: 0.93
